In [1]:
import os

In [2]:
%pwd

'a:\\Python Projects\\Tutorials\\text-summarizer\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'a:\\Python Projects\\Tutorials\\text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import create_directories, read_yaml

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artefacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_tranformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_transformation_config

In [8]:
from text_summarizer.logging import logger
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

[2024-02-13 13:55:01,171: INFO: config: PyTorch version 2.2.0 available.]


c:\Users\ashut\anaconda3\envs\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class DataTranformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config,
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length = 1024, truncation = True)
        
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encoding['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.data_path)
        data_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched= True)
        data_samsum_pt.save_to_disk(os.path.join(self.root_dir, "samsum_dataset"))

In [16]:
try:
    config = ConfigurationManager()
    data_tranformation_config = config.get_data_transformation_config()
    data_tranformation = DataTranformation(config= data_tranformation_config)
    data_tranformation.convert()
except Exception as e:
    raise e

[2024-02-13 13:59:42,504: INFO: common: YAML file config\config.yaml loaded sucessfully]
[2024-02-13 13:59:42,505: INFO: common: YAML file params.yaml loaded sucessfully]
[2024-02-13 13:59:42,506: INFO: common: Created directory at: artefacts]
[2024-02-13 13:59:42,507: INFO: common: Created directory at: artefacts/data_tranformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\ashut\anaconda3\envs\textS\Lib\site-packages\transformers\tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 87920.99 examples/s]
